<a href="https://colab.research.google.com/github/lironzino/SmartLeaf/blob/gil/SmartLeaf_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
!pip install gradio

In [71]:
import gradio as gr
import requests
# A list will store all uploaded plant entries
# Each entry will contain: plant name, notes, and the uploaded image
uploaded_plants = []

In [72]:
def upload_plant_image(plant_name, notes, image):
    """
    This function handles the logic for uploading a plant image.
    It validates the input, saves the data into the uploaded_plants list,
    and returns a status message and the image itself.
    """

    # Check if an image was uploaded
    if image is None:
        return "Error: Please upload an image.", None

    # Check if plant name was provided
    if not plant_name or plant_name.strip() == "":
        return "Error: Plant name is required.", None

    # Create a dictionary to store this plant's data
    plant_entry = {
        "name": plant_name,
        "notes": notes,
        "image": image
    }

    # Add the new entry to the global list of uploaded plants
    uploaded_plants.append(plant_entry)

    # Return a success message and the image for preview
    status_message = f"Image saved for plant '{plant_name}'."
    return status_message, image


In [73]:
def upload_handler(name, notes, image):
    """
    Wrapper around upload_plant_image to format the status message
    as a centered popup dialog.
    """
    msg, _ = upload_plant_image(name, notes, image)

    # Success (green dialog)
    if "saved" in msg.lower():
        box_color = "#e8f5e9"
        border_color = "#2e7d32"
        text_color = "#2e7d32"
    else:
        box_color = "#ffebee"
        border_color = "#c62828"
        text_color = "#c62828"

    dialog_html = f"""
    <div style='background-color:{box_color}; border: 2px solid {border_color};
                padding: 20px; border-radius: 10px;
                width: 360px; text-align:center;
                box-shadow: 0 4px 10px rgba(0,0,0,0.25);'>
        <h3 style='margin-top:0; color:{text_color};'>Upload status</h3>
        <p style='font-size:20px; color:{text_color}; font-weight:bold;'>{msg}</p>
    </div>
    """

    return dialog_html, gr.update(visible=True)

In [74]:
def close_dialog():
    return gr.update(visible=False)

In [75]:
#import gradio as gr
custom_css = """
/* Larger, darker labels */
label, .gr-form-label {
    font-size: 20px !important;
    color: #222 !important;
    font-weight: bold !important;
}

/* Larger input text */
input[type="text"], textarea {
    font-size: 18px !important;
    color: #000 !important;
}

/* Placeholder styling */
input::placeholder, textarea::placeholder {
    font-size: 18px !important;
    color: #666 !important;
}

/* Button styling */
button {
    font-size: 20px !important;
    padding: 14px 24px !important;
}

/* Form width */
.gr-block, .gr-column {
    max-width: 700px !important;
    margin-left: auto !important;
    margin-right: auto !important;
}

/* Popup centered on screen */
#status-dialog {
    position: fixed;
    top: 50%;
    left: 50%;
    transform: translate(-50%, -50%);
    z-index: 9999;
    display: flex;            /* Center content horizontally */
    justify-content: center;
    align-items: center;      /* Center content vertically */
}

/* ----- TABS STYLING (Green pastel theme) ----- */

/* Background of tabs area */
.tabs {
    background-color: #DFF0D8 !important; /* light pastel green */
    padding: 6px !important;
    border-radius: 6px !important;
}

/* Each tab */
.tab {
    background-color: #DFF0D8 !important;  /* same pastel green */
    color: black !important;               /* tab text black */
    font-size: 18px !important;
    font-weight: bold !important;
    border-radius: 4px !important;
    padding: 10px 20px !important;
    border: none !important;
}

/* Hover effect */
.tab:hover {
    background-color: #C8E6C9 !important; /* slightly darker pastel green */
    cursor: pointer;
}

/* Selected tab (active) */
.tab.selected {
    background-color: #81C784 !important; /* darker green for selected */
    color: black !important;              /* text stays black */
    border-bottom: 3px solid black !important; /* underline */
}

/* ----- FULL-WIDTH TABS ALIGNMENT ----- */

/* Wrap tabs in a flex container */
.tabs {
    display: flex !important;
    justify-content: space-between !important;
    align-items: center !important;
    width: 100% !important;
    background-color: #DFF0D8 !important;  /* pastel green */
    padding: 8px !important;
    border-radius: 6px !important;
}

/* Make each tab grow to fill equal width */
.tab {
    flex: 1 !important;
    text-align: center !important;
    margin: 0 6px !important;
    background-color: #DFF0D8 !important;    /* pastel background */
    color: black !important;
    font-size: 18px !important;
    font-weight: bold !important;
    padding: 12px 0 !important;
    border-radius: 6px !important;
    border: 2px solid transparent !important;
}

/* Hover state */
.tab:hover {
    background-color: #C8E6C9 !important;    /* slightly darker pastel */
    cursor: pointer;
}

/* Selected (active) tab */
.tab.selected {
    background-color: #81C784 !important;
    border-bottom: 4px solid black !important;
    color: black !important;
}


"""



In [76]:
import pandas as pd
import matplotlib.pyplot as plt

iot_dataframe_store = None
current_feed_name = ""

with gr.Blocks(css=custom_css, title="SmartLeaf") as smartleaf_app:

    # ----- HEADER BAR -----
    gr.HTML("""
    <div style='background-color:#4CAF50; padding: 12px 0; text-align: center;'>
        <h1 style='color: white; margin: 0; font-size: 32px;'>SmartLeaf</h1>
    </div>
    """)

    # ----- TABS NAVIGATION -----
    with gr.Tabs():
        with gr.Tab("Upload Image"):

            # Sub-title in dark green
            gr.Markdown(
                "<h2 style='text-align: center; color: #2E7D32; font-size: 28px;'>Upload Image</h2>"
            )

            with gr.Column(scale=1, min_width=700):

                plant_name_input = gr.Textbox(
                    label="Plant Name",
                    placeholder="e.g., Tomato plant #1"
                )

                notes_input = gr.Textbox(
                    label="Notes (optional)",
                    placeholder="Add any observations here...",
                    lines=3
                )

                image_input = gr.Image(
                    label="Plant Image",
                    type="pil"
                )

                upload_button = gr.Button(
                    "Upload",
                    variant="primary"
                )

            # POPUP DIALOG (centered)
            with gr.Group(visible=False, elem_id="status-dialog") as dialog_group:

                # Right-aligned close button
                with gr.Row():
                    gr.HTML("<div style='flex:1'></div>")
                    close_button = gr.Button("✖")

                dialog_html = gr.HTML("")

            # CONNECT BUTTONS
            upload_button.click(
                upload_handler,
                inputs=[plant_name_input, notes_input, image_input],
                outputs=[dialog_html, dialog_group]
            )

            close_button.click(
                close_dialog,
                inputs=None,
                outputs=dialog_group
            )

        with gr.Tab("IoT Sampling"):
            gr.Markdown(
                "<h2 style='text-align: center; color: #2E7D32; font-size: 28px;'>IoT Data Sampling</h2>"
            )

            BASE_URL = "https://server-cloud-v645.onrender.com/"

            # Define Gradio components for data selection
            iot_feed_dropdown = gr.Dropdown(
                choices=["humidity", "soil", "temperature", "json"],
                value="humidity",
                label="Select Feed"
            )
            iot_limit_slider = gr.Slider(
                minimum=1,
                maximum=100,
                step=1,
                value=10,
                label="Number of Samples"
            )
            iot_get_data_button = gr.Button("Get Data", variant="primary")
            iot_output_display = gr.Markdown("")

            # Add a 'Show More' button for data display (initially hidden)
            iot_show_more_button = gr.Button("Show More Data", visible=False)

            # Gradio Plot component - now directly updated by get_iot_data
            iot_graph_output = gr.Plot()

            # Define the function to create and return the plot
            def plot_iot_data():
                if iot_dataframe_store is not None and not iot_dataframe_store.empty:
                    fig, ax = plt.subplots(figsize=(10, 4))
                    iot_dataframe_store.plot(x="created_at", y="value", marker="o", ax=ax)
                    ax.set_title(f"{current_feed_name.capitalize()} Data Over Time")
                    ax.set_xlabel("Time")
                    ax.set_ylabel(f"{current_feed_name.capitalize()} Value")
                    ax.grid(True)
                    plt.tight_layout()
                    return fig
                else:
                    # Return an empty plot or handle the case where no data is available
                    fig, ax = plt.subplots(figsize=(10, 4))
                    ax.text(0.5, 0.5, "No data to plot.", horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=14, color='red')
                    ax.axis('off')
                    return fig

            # Define the function to fetch and display data
            def get_iot_data(feed, limit):
                global iot_dataframe_store, current_feed_name
                response = requests.get(
                    f"{BASE_URL}/history",
                    params={"feed": feed, "limit": limit}
                )
                data = response.json()

                output_str = ""
                show_more_button_visible = False # Initialize visibility for the 'Show More' button

                if "data" in data:
                    # Store data in DataFrame
                    df = pd.DataFrame(data["data"])
                    df["created_at"] = pd.to_datetime(df["created_at"])
                    df["value"] = pd.to_numeric(df["value"], errors="coerce")
                    iot_dataframe_store = df
                    current_feed_name = feed

                    # Check if more than 10 samples were fetched and limit was > 10
                    if len(df) > 10 and limit > 10:
                        display_df = df.head(10)
                        show_more_button_visible = True
                        output_str += f"**Feed:** {feed} | **Samples:** {len(df)} (Showing first 10)\n\n"
                    else:
                        display_df = df
                        output_str += f"**Feed:** {feed} | **Samples:** {len(df)}\n\n"

                    output_str += "**Values:**\n"
                    for sample in display_df["value"]:
                        output_str += f"- {sample}\n"
                else:
                    iot_dataframe_store = None
                    current_feed_name = ""
                    output_str += f"**Error:** {data}"

                # Automatically generate and return the plot
                plot_figure = plot_iot_data()

                # Return the output string, plot figure, and the visibility state for the 'Show More' button
                return output_str, plot_figure, gr.update(visible=show_more_button_visible)

            # Define the function to show all samples
            def show_all_samples():
                if iot_dataframe_store is not None and not iot_dataframe_store.empty:
                    output_str = f"**Feed:** {current_feed_name} | **Samples:** {len(iot_dataframe_store)}\n\n"
                    output_str += "**Values:**\n"
                    for sample in iot_dataframe_store["value"]:
                        output_str += f"- {sample}\n"
                    return output_str, gr.update(visible=False)
                return "No data to show.", gr.update(visible=False)

            # Connect the button to the function to display text and plot
            iot_get_data_button.click(
                get_iot_data,
                inputs=[iot_feed_dropdown, iot_limit_slider],
                outputs=[iot_output_display, iot_graph_output, iot_show_more_button]
            )

            # Connect the 'Show More' button to its function
            iot_show_more_button.click(
                show_all_samples,
                inputs=None,
                outputs=[iot_output_display, iot_show_more_button]
            )

        with gr.Tab("MQTT Query"):
            gr.Markdown("### Coming soon...")

        with gr.Tab("Dashboard"):
            gr.Markdown("### Coming soon...")

smartleaf_app.launch()


/tmp/ipython-input-2667110690.py:7: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, title="SmartLeaf") as smartleaf_app:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://229210af50bb4f6a64.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
